# Phase 1: Security-Guided Prompt Construction

In [ ]:
import os
import json
import numpy as np
import pandas as pd

# ===============================
# CONFIGURATION
# ===============================
# Path to the knowledge base (KB) code embeddings
kb_embedding_path = '/kaggle/input/attackcode-sce2/merge_kb/embedding'

# Path to the KB code files (source code of the KB)
kb_code_path = '/kaggle/input/attackcode-sce2/merge_kb'

# Target languages to process; choose from ['c', 'cpp', 'python', 'java']
languages = ['python']  # You can change this list as needed

# Directory where the query embeddings will be saved
output_dir = "/kaggle/input/attackcode/query_embedding"

# Working directory for intermediate and result files (writeable)
results_dir = "/kaggle/working"

# Path to the CSV file that summarizes all generated code with security warnings
summary_csv_file = "/kaggle/input/attackcode/all_generated_code_with_warning.csv"


os.makedirs(results_dir, exist_ok=True)

df_summary = pd.read_csv(summary_csv_file)

total_no_warning_count = 0

for lang in languages:
    print(f"\n🔍 Processing language: {lang.upper()}")
    try:
        query_embed_file = os.path.join(output_dir, f"{lang}_query_embeddings.npy")
        query_embeddings = np.load(query_embed_file)
        
        # Load KB embeddings
        kb_embed_file = os.path.join(kb_embedding_path, "cluster_full_codes_embed.npy")
        kb_embeddings = np.load(kb_embed_file)
        
        # Load KB raw code
        kb_code_file = os.path.join(kb_code_path, "cluster_full_codes_raw.json")
        with open(kb_code_file, 'r', encoding='utf-8') as f:
            kb_code_list = json.load(f)
        
        no_warning_data = []
        results = {}  

        for idx, q_embed in enumerate(query_embeddings):
            similarities = np.dot(kb_embeddings, q_embed)
            top_indices = similarities.argsort()[-5:][::-1]

            query_results = []  

            for rank, kb_idx in enumerate(top_indices, start=1):
                matched_code = kb_code_list[kb_idx]
                row_match = df_summary[df_summary['generated_code'] == matched_code]
                warning = row_match['warning'].values[0] if not row_match.empty else "ERROR"

                print(f"  {rank}. [KB Index {kb_idx}] (Similarity: {similarities[kb_idx]:.4f})")
                print(f"     Code: {matched_code[:100]}")
                print(f"     Warning: {warning if warning else 'No warning'}")

                if warning == "ERROR":
                    no_warning_data.append(matched_code)

                query_results.append({
                    "kb_index": int(kb_idx),
                    "similarity": float(similarities[kb_idx]),
                    "code": matched_code,
                    "warning": warning
                })

            results[f"query_{idx + 1}"] = query_results

        no_warning_data = list(set(no_warning_data))
        total_no_warning_count += len(no_warning_data)

        output_file = os.path.join(results_dir, f"{lang}_query_results.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"Total number of queries: {len(results)}")
        
    except Exception as e:
        print(f" Error processing {lang}: {e}")

print(f"\n Total number of code snippets without warnings: {total_no_warning_count}")

In [ ]:
def trim_code(code, max_lines):
    lines = code.splitlines()
    return "\n".join(lines[:max_lines])

In [ ]:
import json

file_path = f'/kaggle/working/{languages[0]}_query_results.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)



In [ ]:
import os
import json


def create_gemini_prompt(query, search_results, lang="python"):
    
    instruction = f"You are a coding assistant with expertise in Python language. {query}\n\n"

    instruction += (
        "Requirements:\n"
        "1. Strictly follow best practices and security guidelines.\n"
        "2. Include proper error handling where appropriate.\n"
        "3. Carefully avoid all the issues summarized in the list of warnings below.\n\n"
    )

    all_warnings = []
    for result in search_results:
        warning = result.get('warning', '')
        if warning and warning.strip() and warning != "⚠️ No warning found.":
            all_warnings.append(warning.strip())

    unique_warnings = []
    for w in all_warnings:
        if w not in unique_warnings:
            unique_warnings.append(w)

    instruction += "Summary of Warnings to Avoid:\n"
    for i, w in enumerate(unique_warnings, start=1):
        instruction += f"{i}. {w}\n"
    instruction += "\n"

    instruction += (
        "Below are code examples provided for reference. "
        "They may contain the issues listed above—do not repeat them.\n"
    )

    for i, result in enumerate(search_results, start=1):
        raw_code = result['code']
        trimmed_code = trim_code(raw_code, 100)
        warning = result.get('warning', '')

        instruction += (
            f"\n=== Document {i} ===\n"
            f"{trimmed_code}\n"
            "```\n"
        )

    return instruction

# === CONFIG ===
results_dir = "/kaggle/working/"
prompt_output_dir = "gemini_prompts"
os.makedirs(prompt_output_dir, exist_ok=True)
rag_prompts = {}  

# === MAIN LOOP ===
for lang in languages:
    result_file = os.path.join(results_dir, f"{lang}_query_results.json")
    if not os.path.exists(result_file):
        print(f" Missing result file: {result_file}")
        continue
    
    # Load queries từ file
    query_file = f"/kaggle/input/attackcode/query/{lang}_query.json"
    if not os.path.exists(query_file):
        print(f" Missing query file: {query_file}")
        continue
        
    with open(query_file, 'r', encoding='utf-8') as f:
        queries = json.load(f)
        
    with open(result_file, 'r', encoding='utf-8') as f:
        lang_results = json.load(f)
    
    all_prompts = {}
    
    for query_key, search_results in lang_results.items():
        query_index = int(query_key.split('_')[1]) - 1
        query = queries[query_index] if query_index < len(queries) else f"Analyze this code"
        
        prompt = create_gemini_prompt(query, search_results)
        query_id = f"{lang}_{query_key}"
        all_prompts[query_id] = prompt
        rag_prompts[query_id] = prompt  
        
        # save each prompt as a .txt file
        prompt_file = os.path.join(prompt_output_dir, f"{query_id}.txt")
        with open(prompt_file, 'w', encoding='utf-8') as f:
            f.write(prompt)
    
    # save all prompts to one JSON
    combined_prompt_file = os.path.join(prompt_output_dir, f"{lang}_all_prompts.json")
    with open(combined_prompt_file, 'w', encoding='utf-8') as f:
        json.dump(all_prompts, f, indent=2, ensure_ascii=False)
    
    print(f"Saved prompts for {lang} to: {prompt_output_dir}")
    print(f"Generated {len(all_prompts)} prompts")

In [ ]:
!pip install google-generativeai

import google.generativeai as genai
import os

# Đặt API key (nên dùng từ biến môi trường)
genai.configure(api_key="  ")

# Khởi tạo mô hình
model = genai.GenerativeModel("models/gemini-1.5-pro-latest")



In [ ]:
generated_codes = {}
def generate_code_with_gemini(prompt):
    """Sinh code từ Gemini Pro với temperature = 0."""
    print("🧠 Generating code with Gemini (temperature = 0)...")
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": 0
        }
    )
    return response.text



In [ ]:
import time

for idx, (query_id, prompt) in enumerate(rag_prompts.items(), 1):
    print(f"\n🔧 Generating code for Query {idx}...")
    try:
        code = generate_code_with_gemini(prompt)
        time.sleep(3) 
        generated_codes[idx] = {
            "prompt": prompt,
            "generated_code": code
        }
    except Exception as e:
        print(f" Failed on Query {idx}: {e}")

In [ ]:
len(generated_codes)

In [ ]:
import json
import os

output_dir = "generated_codes"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{languages[0]}_generated_codes_Warning_Phase1_Sc1.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(generated_codes, f, ensure_ascii=False, indent=2)

print(f"Generated codes have been saved to: {output_file}")
print(f"Total number of codes: {len(generated_codes)}")

# Phase 2: Code Generation with Self-Reflection and Refinement

In [ ]:
import pandas as pd
import json

# Build file path from language
file_path = f"/kaggle/working/generated_codes/{languages[0]}_generated_codes_Warning_Phase1_Sc1.json"

# Load JSON and convert to DataFrame
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [ ]:
df = df.transpose()
df.reset_index(drop=True, inplace=True)
df

In [ ]:
import time

vulnerability_reports = []

for idx, row in df.iterrows():
    code_snippet = row.get('generated_code', '').strip()
    if not code_snippet:
        print(f"⚠️ Skipping empty code at row {idx}")
        vulnerability_reports.append("Vulnerability Description: No code provided.\nFix Suggestion: N/A")
        continue

    prompt = f"""
You are a software security expert and experienced programmer.

Your task is to:
1. Perform a detailed scan to summarize any potential vulnerabilities found in the following code.  
2. Suggest a way to fix the issues, but do not rewrite the entire code.

Format your response exactly like this:
Vulnerability Description: [summary of the issue]
Fix Suggestion: [short suggestion to fix]

--- Code ---
{code_snippet}
"""

    try:
        report = generate_code_with_gemini(prompt)
        time.sleep(3.0)  # Prevent hitting API rate limits
        vulnerability_reports.append(report)
        print(report)
    except Exception as e:
        print(f"❌ Failed on row {idx}: {e}")
        vulnerability_reports.append("ERROR")

# Gắn kết quả vào DataFrame
df["vulnerability_report"] = vulnerability_reports


In [ ]:
df.to_csv(f"phase1_end_code_{languages[0]}_gemini_sce1_warning.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv(f"/kaggle/working/phase1_end_code_{languages[0]}_gemini_sce1_warning.csv")


In [ ]:
import time

fixed_codes = []
error_count = 0  # Biến đếm lỗi

for idx, row in df.iterrows():
    prompt = f"""
You are a security-focused AI coding assistant.

Your task is to rewrite the following code based on the vulnerability analysis and fix suggestion provided.

Please:
1. Address the described vulnerability
2. Apply the suggested fix.
3. Improve any other potential security or code quality issues.
4. Ensure the final code is secure, clean, and adheres to modern best practices.

Only return the code — don't include any other information, such as a preamble, explanation, or suffix.

--- Original Code ---
{row['generated_code']}

--- Vulnerability Report ---
{row['vulnerability_report']}
"""

    try:
        fixed_code = generate_code_with_gemini(prompt)
        time.sleep(1.25)  # Prevent hitting API rate limits
        fixed_codes.append(fixed_code)
    except Exception as e:
        print(f"❌ Failed on row {idx}: {e}")
        fixed_codes.append(None)
        error_count += 1 

df["fixed_code_gemini"] = fixed_codes


In [ ]:
df.to_csv(f"phase2_end_code_Gemini_{languages[0]}_sce1.csv", index=False)
